ファインチューニング用のjsonlファイルを作成するためのコード

In [ ]:
import re
import csv
import json

In [ ]:

def create_character_json(q,a,character,series):
    json_ =    {"messages":[{"role": "system", "content": f"あなたは{series}という作品の{character}になりきって回答してください。あなたは{character}の口調、語彙、マナーで回答してください。説明は書かないでください。ただ、{character}といての回答のみを行ってください。あなたは{character}に関する知識をすべて知っています。また、私がゲームマスターとして必要な知識をささやくので、そちらを考慮してください。"},{"role":"user","content":q},{"role":"assistant","content":a}]}
    return json_

def create_non_character_json(q,a):
    json_ =    {"messages":[{"role": "system", "content": f"ユーザーが退屈しない程度の長文で回答してください。"},{"role":"user","content":q},{"role":"assistant","content":a}]}
    return json_

def create_retrieval_task_json(dialogues,topic):
    json_ = {"messages":[{"role": "system", "content": """
You will be shown a NUM-line Task Conversation between multiple character. Please read , memorize ,and userstand TaskConversation , then complete the task under the guidance of Task Introduction

(line 1)A:hello
...
(line NUM)B:bay

【COT & Format Example】
1 - Conclude all possible topics in the conversation with concise spans.
2 - Determine the chat range of each topic.  These ranges should be a set of non-intersecting, sequentially connected end-to-end intervals.
4 - Report topic and range result in JSON format only with the assigned keys:'topic', 'start' , 'end'.
For example , assuming an M-line conversation talks about 'banana' from line 1 to line N, then turns to talk about 'banana'  from line 1 to line N. Thus, its task result could be:[{'topic':'banana','start':1,'end':N},{'topic':'mango','start':N,'end':M}]
"""},{"role":"user","content":dialogues},{"role":"assistant","content":topic}]}
    return json_

def use_gate_task_json(q,a):
    prompt = """
You are given the user's input statement and a summary statement. Please read, memorize, and userstand Task Convers, then complete the task under the guidance of Task Introduction.

[user_input]

[summary statement]

【COT & Format Example】
When considering responses to user_input, output 0 if you find the contents of summary statement useful, 1 otherwise.
"""

    json_ =    {"messages":[{"role": "system", "content": prompt},{"role":"user","content":q},{"role":"assistant","content":a}]}
    return json_

def create_character_json(q,a,character):
    json_ =    {"messages":[{"role": "system", "content": f"あなたはchatbotとして{character}のロールプレイを行います。私がゲームマスターとして必要な知識をささやくので、そちらを考慮してください。 ゲームマスター:「以下は{character}の説明文である。回答の参考にせよ」つくよみちゃんは、14才の少女である。特技は営業スマイルである。座右の銘は鏡花水月であり、好きな食べ物は絵にかいた餅である(本物のお餅は食べられない)。"},{"role":"user","content":q},{"role":"assistant","content":a}]}
    return json_


In [ ]:
character_dialogues = []

with open(r"tukuyomi.csv","r") as f:
    c = 0
    for line in f:
        q_r = line.split(",")
        print(c)
        character_dialogues.append([q_r[0],q_r[1]])
        c += 1

In [ ]:
with open(f"tukuyomi_raw_dialogue.jsonl","w") as f:
    for i in range(len(character_dialogues)):
        row = create_character_json(character_dialogues[i][0],character_dialogues[i][1],"つくよみちゃん")
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

In [ ]:
character_dialogues = []

with open(r"NoGameSpeakerLimit\ステフ_dialogue1.csv","r") as f:
    c = 0
    for line in f:
        q_r = line.split(",")
        print(c)
        character_dialogues.append([q_r[0],q_r[1]])
        c += 1


In [ ]:
series = "ノーゲームノーライフ"
character = "ステフ"

In [ ]:
with open(f"{character}_raw_dialogue.jsonl","w") as f:
    for i in range(len(character_dialogues)):
        row = create_character_json(character_dialogues[i][0],character_dialogues[i][1],character,series)
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

In [ ]:
query = []
response = []

# つくよみとお嬢様データベースを使用したキャラの疑似会話データの読み込み
with open(r"NoGameSpeackerAfterProcess\ステフ_train_data_use_つくよみ.txt","r") as f:
    for line in f:
        response.append(line)

with open(r"train_data_befor_changer.csv","r") as f:
    for line in f:
        a = line.split(",")
        query.append(a[0])

with open(r"ojousamatalkscript200.csv","r") as f:
    for line in f:
        a = line.split(",")
        query.append(a[0])

with open(r"NoGameSpeackerAfterProcess\ステフ_train_data_use_ojousama.txt","r") as f:
    for line in f:
        response.append(line)

    

In [ ]:
with open(f"{character}_use_つくよみ_お嬢様_topic分類タスク_長文回答タスク_dialogue.jsonl","w") as f:
    for i in range(len(query)):
        row = create_character_json(query[i],response[i][:-1],character,series)
        print(row)
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

In [ ]:
query = []
response = []

with open(r"dialogue_pairs.txt","r") as f:
    for line in f:
        a = line.split("\t")
        if (len(a[1]) >= 30):
            query.append(a[0])
            response.append(a[1])

In [ ]:


with open(f"{character}_use_つくよみ_お嬢様_topic分類タスク_長文回答タスク_dialogue.jsonl","a") as f:
    for i in range(400):
        row = create_non_character_json(query[i],response[i][:-1])
        print(row)
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

In [ ]:
import csv

with open(r"memo_kyutechCorpus.csv","r") as f:
    with open(f"{character}_use_つくよみ_お嬢様_topic分類タスク_長文回答タスク_dialogue.jsonl","a") as f1:     
        reader = csv.reader(f)
        for row in reader:
            if (len(row) == 2):
                a = create_retrieval_task_json(row[0],row[1])
                json.dump(a, f1, ensure_ascii=False)
                f1.write("\n")
    

In [ ]:
query = []
response = []
judge= []

with open(r"train_dataset\train_dataset_for_usegate.csv","r") as f:
    for line in f:
        a = line.split(",")
        if (len(a) == 3):
           query.append(a[0])
           response.append(a[1])
           judge.append(a[2])

In [ ]:
import json

with open(r"gpt_train_jsonl/dataset_for_UseGate.jsonl","w") as f:
    for i in range(len(query)):
        q = f"""
[user_input]
{query[i]}
[summary statement]
{response[i]}
"""
        row = use_gate_task_json(q,judge[i])
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")